In [1]:
import os
os.environ["XDG_CACHE_HOME"] = "/home/olab/tomerronen1/xdg_cache/"
from bert_score import BERTScorer
scorer = BERTScorer(model_type="microsoft/deberta-xlarge-mnli", lang="en", rescale_with_baseline=True)

Some weights of the model checkpoint at microsoft/deberta-xlarge-mnli were not used when initializing DebertaModel: ['pooler.dense.weight', 'classifier.bias', 'pooler.dense.bias', 'classifier.weight']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
from pathlib import Path
import jsonlines
dump_dir = Path("/home/olab/tomerronen1/git_repos/last_projects_playground/confidence_estimation/openai_dump")
for subdir in dump_dir.iterdir():
    with jsonlines.open(subdir / "metrics.jsonl", 'r') as reader:
        metrics_list = list(reader.iter())
    with jsonlines.open(subdir / "sentences.jsonl", 'r') as reader:
        sentences_list = list(reader.iter())

    if "bertscore_f1" not in metrics_list[0]:
        print(f"Calculating BertScore for {subdir.name}")

        candidates = [sentences["pred_text"] for sentences in sentences_list]
        references = [sentences["target_text"] for sentences in sentences_list]
        precisions, recalls, f1s = scorer.score(candidates, references)
        precisions, recalls, f1s = precisions.tolist(), recalls.tolist(), f1s.tolist()
        metrics_list = [
            {"bertscore_precision": precision, "bertscore_recall": recall, "bertscore_f1": f1, **metrics}
            for precision, recall, f1, metrics
            in zip(precisions, recalls, f1s, metrics_list)
        ]
        
        with jsonlines.open(subdir / "metrics.jsonl", 'w') as writer:
            writer.write_all(metrics_list)


Calculating BertScore for wmt19_de_to_en__text-davinci-002__200_examples
